In [ ]:
%pip install -U spacy
%pip install -U wordcloud
%pip install -U keras
%pip install -U seaborn
%pip install -U tensorflow
%pip install -U matplotlib
%pip install -U pandas
%pip install -U scikit-learn


In [2]:
import pandas as pd
import seaborn as sns
import spacy
import re
from tqdm import tqdm
from wordcloud import WordCloud
import matplotlib.pyplot as P
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import tensorflow as tf

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import keras
import keras
from keras.models import Sequential
from keras.models import load_model
from keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences, to_categorical
tqdm.pandas()


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
TRAINDATA_FILE_PATH = '../dataSample/train_dataset.csv'
print("Reading data from file: ", TRAINDATA_FILE_PATH)



In [ ]:
TRAINDATA_FILE_PATH = '../dataSample/train_dataset.csv'
print("Reading data from file: ", TRAINDATA_FILE_PATH)

names=['text', 'label']
try:
    df = pd.read_csv(TRAINDATA_FILE_PATH, names=names)
    df.sample(5)
    df.info()
    df.groupby('text').nunique()
    df['label'] = df['label'].map({0: 'Negative', 1: 'Positive'})
    sns.countplot(x='label',data=df)
    print("Data read successfully.")
except FileNotFoundError:
    print(f"Error: The file {TRAINDATA_FILE_PATH} was not found.")
except pd.errors.EmptyDataError:
    print("Error: The file is empty.")
except Exception as e:
    print(f"An error occurred: {e}")


In [6]:
## Importing old jrd file

def clean_data(data, re_letters):
    data['text'] = data['text'].apply(lambda x: x.lower())
    data['text'] = data['text'].apply(lambda x: re_letters.sub('', x))


def remove_stopwords(text):
    return ' '.join(filter(lambda x: x not in nlp.Defaults.stop_words, text.split()))


def lemmatize(text):
    return ' '.join([x.lemma_ for x in nlp(text)])


def build_wordcloud(data, label_type, max_words, c_width, c_height):
    text = ' '.join(data['text'][data['label'] == label_type])
    wc = WordCloud(max_words=max_words, width=c_width, height=c_height, collocations=False).generate(text)
    return wc.to_image()

y = 5

In [ ]:
data=df[['text','label']]
re_letters=re.compile(r"[^a-zA-Z\s']")

clean_data(data, re_letters)
data.sample(5)

In [8]:
nlp = spacy.load('en_core_web_md',disable=['ner', 'parser'])
nlp.add_pipe('sentencizer')
nlp.Defaults.stop_words.add("game")
nlp.Defaults.stop_words.add("play")
nlp.Defaults.stop_words.add("t")



In [ ]:
data['text']=data['text'].apply(remove_stopwords)
data.sample(5)



In [ ]:
data['text']=data['text'].progress_apply(lemmatize)
data.sample(5)

In [ ]:
build_wordcloud(data, 'Negative', 1000, 1600, 800)

In [ ]:
build_wordcloud(data, 'Positive', 1000, 1600, 800)

In [ ]:
data
max_words = 5000
max_len = 500

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.text)
sequences = tokenizer.texts_to_sequences(data.text)
reviews = pad_sequences(sequences, maxlen=max_len)
print(reviews)

In [12]:
data['label'] = data['label'].map({'Negative': 0, 'Positive': 1})
labels=to_categorical(data.label,num_classes=2)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.1, stratify=labels)

In [ ]:
model3 = Sequential()
model3.add(layers.Embedding(max_words, 40, input_length=max_len)) # optimize hyper parameters
model3.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model3.add(layers.Dense(2,activation='softmax'))

model3.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

history = model3.fit(X_train,
                     y_train,
                     epochs=8,
                     validation_data=(X_test, y_test))

In [ ]:
names=['text', 'label']
df_test = pd.read_csv('drive/MyDrive/SUML_data/test_dataset.csv', names=names)
data_test=df_test[['text','label']]
clean_data(data_test, re_letters)
data_test['text']=data_test['text'].apply(remove_stopwords)
data_test['text']=data_test['text'].progress_apply(lemmatize)
data_test.sample(5)
sns.countplot(x='label',data=data_test)

In [ ]:
#load the pre-trained model
current_model = load_model('drive/MyDrive/SUML_data/18_01_model.h5')


In [ ]:
# Make predictions on new text data using the pre-trained model
new_sequences = tokenizer.texts_to_sequences(data_test.text)
new_data = pad_sequences(new_sequences, maxlen=max_len)
new_predictions = current_model.predict(new_data)

score = 0
print(len(data_test))

# Print the predicted labels for new text data
for i, prediction in enumerate(new_predictions):
    if prediction[1] > prediction[0]:
      label = 1
    else:
      label = 0

    if (data_test.label[i] == label):
      score = score + 1

print(score)
print(score / len(data_test))

SAVING THE MODEL USING PICKLE

In [16]:
import pickle

In [ ]:
filename = 'model30122024pickle.sav'
pickle.dump(model3, open(filename, 'wb'))

# save the model
model3.save('model30122024keras.keras')
model3.save('model30122024.h5')

In [ ]:
current_model = pickle.load(open('model30122024pickle.sav', 'rb'))

In [ ]:
# Make predictions on new text data using the pre-trained model
while True:
    user_input = input("Enter a review (or 'exit' to end): ")

    if user_input.lower() == 'exit':
        break

    # Tokenize and pad the user input
    user_sequence = tokenizer.texts_to_sequences([user_input])
    user_data = pad_sequences(user_sequence, maxlen=max_len)

    # Make predictions for the user input
    user_prediction = current_model.predict(user_data)[0]
    print(user_input)
    print(user_data)
    # Print the predicted label for the user input
    if user_prediction[1] > user_prediction[0]:
        predicted_label = 1
    else:
        predicted_label = 0

    print("Predicted Label:", predicted_label)

In [20]:
#save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    